In [1]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [3]:
df = pd.read_csv("raw_data.csv", index_col= 0) ## de cara a la exportación para no duplicar el index fantasma

In [ ]:
lista_errores = [sameasmonthlyincome,  salary, monthlyincome, monthlyrate]

def cambiar_comas(dato):
    try:
        return float(dato.replace(",", "."))
    except:
        lista_errores.append(dato)
        return np.nan

In [ ]:
lista_col = []

In [ ]:
for col in lista_col:
    print(col)
    df[col] = df[col].apply(cambiar_comas)

In [ ]:
$--> hourlyrate - ya tiene punto y no tiene $ (limpio) HACER round(2) para solo ensenar centimos.
COMMA a PUNTO: performancerating, worklifebalance -- podrian ser int o float

In [ ]:
sameasmonthlyincome,  salary
CAMBIAR comma a punto, quitar $, float: sameasmonthlyincome,  salary, monthlyincome, monthlyrate
,
$ -->daily rate - ya tiene punto y no tiene $ (limpio) HACER round(2) para solo ensenar centimos.